<a href="https://colab.research.google.com/github/pepper29/book_studying_alone_ML-DL/blob/main/CH5_2_CrossValidation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#검증세트
# df 에서 np로 변환시키기
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [ ]:

from sklearn.model_selection import train_test_split
# 훈련&테스트데이터로 나누기
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

# 훈련데이터를 다시 검증데이터로 나누기
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [ ]:
# 훈련세트는 5197개, 테스트세트는 1300개
print(train_input.shape, test_input.shape)

(5197, 3) (1300, 3)


In [ ]:
# 훈련세트에서 20%를 나눴으니 바뀐 훈련세트는 4157개, 검증세트는 1040개이다.
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [ ]:
#모델 만들기
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [ ]:
# 교차검증에 대해 사용해보기
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.01173401, 0.01027465, 0.01383901, 0.02025437, 0.01882148]), 'score_time': array([0.00188589, 0.00185394, 0.00234461, 0.00326943, 0.0018084 ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [ ]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
#훈련세트 섞어서 해보기
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [ ]:
#그리드 서치를 통한 하이퍼파라미터 조정해보기
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease':[0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [ ]:
#5-fold 교차검증 수행해보기 총 5*5 = 25개 모델을 훈련한다.
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [ ]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [ ]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [ ]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [ ]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [ ]:
#조금 더 복잡한 매개변수 조합탐색
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001), #첫번째 매개변수에서 두번째 매개변수까지 세번째 매개변수만큼을 계속 더해서 반복
          'max_depth': range(5, 20, 1), #위와같음
          'min_samples_split': range(2, 100, 10)  #위와같음
          }
# 1번함수에서 9개 + 2번함수에서 15, 3번에서 10개 총 교차검증횟수는 9*15*10=1,350개 이다.

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [ ]:
# 최적파라미터 계산
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [ ]:
# 교차검증 점수계산
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [ ]:
# 랜덤서치
# 시간이 오래걸리는 그리드서치의 단점을 보완한다.
# stats는 균등 샘플링 함수
# radint는 정수, uniform은 실수이다.
from scipy.stats import uniform, randint
rgen = randint(0, 10)
rgen.rvs(10)

array([9, 9, 7, 2, 2, 8, 0, 7, 0, 4])

In [ ]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([108, 100, 104, 105, 111,  97, 102,  83,  89, 101]))

In [ ]:
ugen = uniform(0, 1)
ugen.rvs(10)
# 쉽게 난수발생기라고 생각하면 된다.

array([0.00540881, 0.65105744, 0.26326177, 0.49817483, 0.75807257,
       0.83162918, 0.84452499, 0.61753282, 0.88368497, 0.52173542])

In [ ]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [ ]:
# 랜덤서치 사용해보기
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ed26b584970>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7ed26b584dc0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ed26b587190>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ed26b584640>},
                   random_state=42)

In [ ]:
# 최적 파라미터 확인
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [ ]:
#검증 최대값 확인하기
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [ ]:
#최적파라미터로 테스트데이터 실험하기
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86
